In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from spot_gl.spot_basic import *
from spot_gl.spot_arm import *
from spot_gl.spot_graph_nav import *
from spot_gl.gl_functions import *

In [3]:
#### CONSTANTS ####
obj_search_cam_coords = [{'moveTo':[0.35, 0.4, 0.5], 'lookAt':[0.35, 0.6, -0.5]}, 
                      {'moveTo':[0.6, 0.0, 0.5],  'lookAt':[1.0, 0.0, -0.5]},
                      {'moveTo':[0.35, -0.4, 0.5],'lookAt':[0.35, -0.6, -0.5]}]

In [4]:
sdk, robot, id_client, \
robot_state_client, command_client, \
image_client, graph_nav_client, \
world_object_client, manipulation_api_client = init_robot('192.168.50.3')

In [5]:
lease_client, lease, lease_keep_alive = get_lease(robot)

In [6]:
# Initialize Groundlight API
gl = Groundlight(api_token=os.environ.get("GROUNDLIGHT_API_TOKEN"))
detectors = gl.list_detectors()
can_det = detectors.results[0]

In [1]:
get_batt_info(robot_state_client)

NameError: name 'get_batt_info' is not defined

In [36]:
success = power_on(robot)

In [37]:
bdcrc.blocking_selfright(command_client, timeout_sec = 20)

In [11]:
def obj_search(command_client, image_client, gl, obj_det, verbose = False):
    tree = None
    hand_color = None

    for hand_pos_ind in range(len(obj_search_cam_coords)):

        cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(1.0)
        cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

        move_to_and_look_at(command_client, 
                        obj_search_cam_coords[hand_pos_ind]['moveTo'], 
                        obj_search_cam_coords[hand_pos_ind]['lookAt'])
        time.sleep(0.5)
        hand_color_img, image_response = cap_hand_image(image_client, 'hand_color_image')
        time.sleep(0.1)
        tree = sweep_localize(gl, obj_det, hand_color_img, verbose = verbose)

        if tree is not None:
            print('found OBJ')
            return tree, hand_color_img, image_response
    print('FAILED TO FIND OBJ')
    return None, None, None

In [12]:
# tree, hand_color_img, image_response = obj_search(command_client, image_client, gl, can_det, verbose = True)
# fig, ax = plt.subplots(1,1, figsize = (10,8))
# plot_tree_on_image(ax, tree, hand_color_img)

In [38]:
graph_upload_filepath = "/home/gluser/groundlight/bdspot/spot/loopGraph"
upload_graph_and_snapshots(graph_nav_client, graph_upload_filepath)
success = graph_localize_fiducial(command_client, graph_nav_client, robot_state_client, robot)
waypoint_list = create_waypoint_list(graph_nav_client)

In [41]:
nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[0], cmd_duration = 15)

In [49]:
frame_tree_snapshot = robot.get_frame_tree_snapshot()
cmd = RobotCommandBuilder. \
synchro_trajectory_command_in_body_frame(-3, 
                                         0, 
                                         0,
                                         frame_tree_snapshot = frame_tree_snapshot,
                                         locomotion_hint = 0)
cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)
bdcrc.block_for_trajectory_cmd(command_client, cmd_id, timeout_sec = timeout_sec)

True

In [ ]:
    frame_tree_snapshot = robot.get_frame_tree_snapshot()
    cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(x_meters, y_meters, theta_rad, frame_tree_snapshot = frame_tree_snapshot)
    cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)
    if blocking:
        bdcrc.block_for_trajectory_cmd(command_client, cmd_id, timeout_sec = timeout_sec)

In [19]:
# loop_waypoints = [25, 5, 6, 9, 10, 12, 15, 17, 18, 19, 20, 21, 23, 24]

# for wp in loop_waypoints:
#     print(wp)
#     nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[wp], cmd_duration = 15)
#     wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)

25
5
6
9
10
12
15
17
18
19
20
21
23
24


In [22]:
#loop_waypoints = [25, 5, 6, 9, 10, 12, 15, 17, 18, 19, 20, 21, 23, 24]

def loop_until_grasp(graph_nav_client, 
                     command_client, 
                     image_client, 
                     manipulation_api_client,
                     loop_waypoints = [9, 10, 12],
                     verbose = False):
    while True:
        for waypoint_ind in loop_waypoints:
            nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[waypoint_ind], cmd_duration = 15)
            wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)

            wp_loc_proto = graph_del_get_graph_loc(graph_nav_client)

            tree, hand_color_img, image_response = obj_search(command_client, image_client, gl, can_det, verbose = verbose)
            
            cmd = RobotCommandBuilder.arm_stow_command()
            cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

            if tree is None:
                upload_graph_and_snapshots(graph_nav_client, upload_filepath)
#                 try:
#                     print('attempting to localize with guess')
#                     graph_localize_guess(graph_nav_client, robot_state_client, wp_loc_proto)
#                 except bosdyn.client.graph_nav.RequestFailedError as e:
#                     print('FAILED TO LOCALIZE WITH GUESS, FINDING FIDUCIAL NOW')
                graph_localize_fiducial(command_client, graph_nav_client, robot_state_client, robot)
                continue

            pxLoc = assemble_px_tree(tree)
            pick_vec = geometry_pb2.Vec2(x=pxLoc[0], y=pxLoc[1])
            grasp = manipulation_api_pb2.PickObjectInImage(
                pixel_xy=pick_vec, transforms_snapshot_for_camera=image_response.shot.transforms_snapshot,
                frame_name_image_sensor=image_response.shot.frame_name_image_sensor,
                camera_model=image_response.source.pinhole)#, grasp_params = graspParams)
            grasp_request = manipulation_api_pb2.ManipulationApiRequest(pick_object_in_image=grasp)
            response, success = make_grasp(grasp_request, manipulation_api_client, command_client, graph_nav_client, verbose = verbose)
        
            return response, success, tree, hand_color_img

    

In [27]:
response, success, tree, hand_color_img = loop_until_grasp(graph_nav_client, command_client, image_client, manipulation_api_client)

found OBJ


In [29]:
def get_recycle_coords(world_object_client,):
    request_fiducials = [world_object_pb2.WORLD_OBJECT_APRILTAG]
    fiducial_objects = world_object_client.list_world_objects(
                object_type=request_fiducials).world_objects

    fid = None
    for fiducial_obj in fiducial_objects:
        if fiducial_obj.apriltag_properties.tag_id == 579:
            fid = fiducial_obj

    if fid is None:
        print("COULD NOT FIND RECYCLING CAN RETURNING NONE")
        return None


    fid_tree_snapshot = fid.transforms_snapshot
    frame_tree_snapshot = robot.get_frame_tree_snapshot()

    #get_a_tform_b returns transformation such that a_coords = tform * b_coords

    fid_to_vision = bosdyn.client.frame_helpers.get_a_tform_b(fid_tree_snapshot, frame_a = 'vision', frame_b ='filtered_fiducial_579')
    
    return fid_to_vision

In [30]:
fid_to_vision = get_recycle_coords(world_object_client)

In [31]:
drop_pose_fid_frame_proto = geometry_pb2.SE3Pose(
    position = geometry_pb2.Vec3(x = 0.3, y = 0, z = -0.1), 
    rotation = bosdyn.client.math_helpers.Quat(w = 0, 
                                               x = 0, 
                                               y = -(1/2)**0.5, 
                                               z = (1/2)**0.5).to_proto())
    #rotation = (bosdyn.client.math_helpers.Quat(1, 0, 0, 0).normalize()).to_obj())
    #rotation = geometry_pb2.Quaternion(w = , x = 0, y = 0, z = 0))

drop_pose_fid_frame_se3 = bosdyn.client.math_helpers.SE3Pose.from_proto(drop_pose_fid_frame_proto)

drop_pose_vision_frame_se3 = (fid_to_vision * drop_pose_fid_frame_se3)

In [32]:
print("pitch", drop_pose_vision_frame_se3.rotation.to_pitch())
print("roll", drop_pose_vision_frame_se3.rotation.to_roll())
print("yaw", drop_pose_vision_frame_se3.rotation.to_roll())

pitch 1.4894474512941542
roll 1.388678107933378
yaw 1.388678107933378


In [33]:
body_cmd = RobotCommandBuilder.follow_arm_command()
arm_cmd = RobotCommandBuilder.arm_pose_command(x = drop_pose_vision_frame_se3.x,
                                           y = drop_pose_vision_frame_se3.y,
                                           z = drop_pose_vision_frame_se3.z,
                                           qw = drop_pose_vision_frame_se3.rotation.w,
                                           qx = drop_pose_vision_frame_se3.rotation.x,
                                           qy = drop_pose_vision_frame_se3.rotation.y,
                                           qz = drop_pose_vision_frame_se3.rotation.z,
                                           frame_name = 'vision')
cmd = RobotCommandBuilder.build_synchro_command(body_cmd, arm_cmd)

cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

In [34]:
cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(1.0)
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)
time.sleep(1.0)

cmd = RobotCommandBuilder.arm_stow_command()
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

In [50]:
bosdyn.client.power.safe_power_off(command_client, robot_state_client)

In [19]:
response = lease_client.return_lease(lease)

No lease for resource "body"
    (resuming check-in)
No lease for resource "body"
    (resuming check-in)
No lease for resource "body"
    (resuming check-in)


In [ ]:
### DEMO ENDS HERE ### Helfpul lines below

In [1]:
# nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[9], cmd_duration = 15)

In [90]:
wp_loc_proto = graph_del_get_graph_loc(graph_nav_client)

In [ ]:
def random_cube_drop(command_client):
    randomLookDir = [((np.random.rand() * 2) - 1) * 2, 
                 (((np.random.rand()) * 2) - 1) * 3, 
                 np.random.rand() * (-2)  ]

    move_to_and_look_at(command_client, [0.8,0,-0.35],randomLookDir, True)
    
    time.sleep(0.5)

    cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(1.0)
    cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

    time.sleep(0.5)

    cmd = RobotCommandBuilder.arm_stow_command()
    cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

In [57]:
# move_robot_relative(command_client, robot, 0.5, 0.2, 0)

In [43]:
# upload_graph_and_snapshots(graph_nav_client, upload_filepath)
# response = graphLocalizeGuess(graph_nav_client, wp_loc_proto)

Loading the graph from disk into local storage...
Loaded graph has 27 waypoints and 29 edges
Uploading the graph and snapshots to the robot...


Upload complete! The robot is currently not localized to the map; please localize
